# Fine-tuning ClinicalBERT with MIMIC-CXR data 
- Version: padded
### Access all dataset here: 
https://imperiallondon-my.sharepoint.com/:f:/g/personal/dlc19_ic_ac_uk/EgobSIgJFitCuMdL0Sg6KmABP7qqtibuOz1R1jIZDEX22Q?e=U5CfiK

In [1]:
import os
import sys
import random
import warnings
warnings.filterwarnings("ignore")

#data manupulation libs
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split

#from pandarallel import pandarallel

# Initialization
#pandarallel.initialize()

#string manupulation libs
#string manupulation libs
import re
import string
from string import digits
import spacy
import nltk
from nltk.corpus import stopwords 

#torch libs
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from datasets import Dataset, load_dataset
import torch.nn as nn
import torch.optim as optim

import transformers

from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments

# data manipulations
from pathlib import Path
import uuid
import pydicom

from PIL import Image

import cv2
import matplotlib.pyplot as plt

import collections

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

from transformers import default_data_collator

[nltk_data] Downloading package stopwords to
[nltk_data]     /media/SharedUsers/dlc19/home/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(1)) # use gpu 1

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

### Data Preprocessing - done separately from data loader

In [2]:
# df_train_temp = pd.read_csv('train_raw_reports.csv')
# df_test_temp = pd.read_csv('test_raw_reports.csv')

# #convetring study id to string as it doesn't work as an int
# df_train_temp['study_id']=df_train_temp['study_id'].astype(str)
# df_test_temp['study_id']=df_test_temp['study_id'].astype(str)

In [3]:
# # display raw reports in a dataframe
# df_train_temp.head()

In [4]:
# # preprocessing of the train dataset
# # stop_words = set(stopwords.words('english'))
# def preprocessing(text):
#         cleanedReport = re.sub(r'[^\w\s]','',text)            # remove punctuation (not word characters and whitespace)
#         cleanedReport = re.sub('_', '', cleanedReport)        # remove __ in the report
#         cleanedReport = re.sub(r'[\d-]', '', cleanedReport)   # remove numbers in the report 
#         cleanedReport = re.sub('\n', '', cleanedReport)
        
#         return cleanedReport   
    

# def preprocessDataframe(df):
    
#     i = 0
    
#     for i in range(len(df)):
        
#         preprocessedText = preprocessing(df.at[i, "raw_report"])
    
#         df.at[i,'raw_report'a] = preprocessedText
#         i = i + 1 
#     return df

In [5]:
# # preprocess the raw reports in the dataframe 
# df_train_preprocessed = preprocessDataframe(df_train_temp)
# df_test_preprocessed = preprocessDataframe(df_test_temp)

In [6]:
# df_train_preprocessed.head() # check if that worked

### Removing Stopwords
Don't actually know whether we should be doing this. If we do, need to make our own list (big problem with getting rid of negations and changing meaning).
If you want to use, comment out and use "df_trained_preprocessed_nostopwords" to make csv


In [7]:
# # Example. Notice the reports have different meanings because of removal of words like 'no'
# print(df_train_preprocessed['raw_report'][0])
# words = [word for word in df_train_preprocessed['raw_report'][0].split() if word.lower() not in stop_words ]
# new_text = " ".join(words)

# print()
# print(new_text)

#### Uncomment to remove stopwords

In [8]:
# df_train_preprocessed_nostopwords = df_train_preprocessed.copy()
# l = len(df_train_preprocessed['raw_report'])
# for i in range(0,l):
#     if i%10000 == 0:
#         print(i) # just to check progress - should take 10-15mins, 220,000 reports
#     words = [word for word in df_train_preprocessed['raw_report'][i].split() if word.lower() not in stop_words]
#     new_report = " ".join(words)
#     df_train_preprocessed_nostopwords['raw_report'][i] = new_report

# df_test_preprocessed_nostopwords = df_test_preprocessed.copy()
# l = len(df_test_preprocessed['raw_report'])
# for i in range(0,l):
#     words = [word for word in df_test_preprocessed['raw_report'][i].split() if word.lower() not in stop_words]
#     new_report = " ".join(words)
#     df_test_preprocessed_nostopwords['raw_report'][i] = new_report
    
    

#### Checking:

In [9]:
# print(df_train_preprocessed['raw_report'][10030])
# print()
# print(df_train_preprocessed_nostopwords['raw_report'][10030])
# print()
# print(df_test_preprocessed['raw_report'][2000])
# print()
# print(df_test_preprocessed_nostopwords['raw_report'][2000])

In [10]:
# # if removing stopwords, change these to _nostopwords versions.
# df_train_preprocessed.to_csv('train_preprocessed.csv', index=False)
# df_test_preprocessed.to_csv('test_preprocessed.csv', index=False)

### Loading data in

In [2]:
data_files = {"train": "train_preprocessed.csv", "test": "test_preprocessed.csv"}
reports_dataset = load_dataset("csv", data_files=data_files)

reports_dataset

Using custom data configuration default-cd976364fa85dbba
Reusing dataset csv (/media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['study_id', 'raw_report'],
        num_rows: 222337
    })
    test: Dataset({
        features: ['study_id', 'raw_report'],
        num_rows: 3269
    })
})

In [3]:
print(reports_dataset["train"]["raw_report"][0])
print(type(reports_dataset["train"]["raw_report"][0]))

                                 FINAL REPORT EXAMINATION  CHEST PA AND LAT  INDICATION  F with new onset ascites   eval for infection  TECHNIQUE  Chest PA and lateral  COMPARISON  None  FINDINGS   There is no focal consolidation pleural effusion or pneumothorax  Bilateral nodular opacities that most likely represent nipple shadows The cardiomediastinal silhouette is normal  Clips project over the left lung potentially within the breast The imaged upper abdomen is unremarkable Chronic deformity of the posterior left sixth and seventh ribs are noted  IMPRESSION   No acute cardiopulmonary process
<class 'str'>


### Data Loader - Padding
- Include: loading data, text preprocessing, words frequency check, tokenization, tokens-IDs-conversion 

In [ ]:
padsize = 150

In [4]:
def tokenize_function_padding(dataset):
    result = tokenizer(dataset["raw_report"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
        
    return result

# Use batched=True to activate fast multithreading!
tokenized_datasets = reports_dataset.map(
    tokenize_function_padding, batched=True, remove_columns=["raw_report", "study_id"])

tokenized_datasets

Loading cached processed dataset at /media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-e549f7f84f54e327.arrow
Loading cached processed dataset at /media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-eb4036fc2652fffe.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 222337
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 3269
    })
})

In [5]:
def padding(dataset):
    
    num_items = len(dataset['input_ids']) # to get number of all items in train dataset
        
    while(len(dataset['input_ids']) < padsize):
        dataset['input_ids'].append(0)
        dataset['token_type_ids'].append(0)
        dataset['attention_mask'].append(0)
        dataset['word_ids'].append(0)

    dataset['labels'] = dataset['input_ids'].copy()
    return dataset

padded_dataset = tokenized_datasets.map(padding, batched=False)

Loading cached processed dataset at /media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-1e107e678d4f9a23.arrow
Loading cached processed dataset at /media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-178bf7936c7e7de3.arrow


In [6]:
len(padded_dataset["test"]["input_ids"][0])

300

In [7]:
smaller_dataset = padded_dataset.filter(lambda example: len(example['input_ids'])<=padsize)

Loading cached processed dataset at /media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-30ec09aaec7ad9ad.arrow
Loading cached processed dataset at /media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-e875dd6ef505ad50.arrow


In [8]:
padded_dataset = smaller_dataset

def check_function(dataset):

    lenReport = len(dataset['input_ids'])
    
    if (lenReport > padsize):
        print('yes')
    return dataset

padded_dataset.map(check_function, batched = False)

Loading cached processed dataset at /media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-34cb2502896db7e8.arrow
Loading cached processed dataset at /media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-6999c1b631150c05.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 221006
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3250
    })
})

In [9]:
torch.cuda.empty_cache()

### If loading from JSON files, use this:
- JSON aren't working rn, hence dataloader

In [11]:
# data_files = {
#     "train" : "final_dataset_chunkedtrain.jsonl",
#     "test" : "final_dataset_chunkedtest.jsonl"    
# }

# chunkedDataset = load_dataset("json", data_files = data_files)
# chunkedDataset


# #testing
# data_files_padded = {
#     "train" : "final_dataset_padded_train.jsonl",
#     "test" : "final_dataset_padded_test.jsonl"    
# }
# paddedDataset = load_dataset("json", data_files = data_files_padded)
# chunkedDataset
    

# MASKING

### Report example

In [12]:
# # same report, padded
# example = padded_dataset["train"]["input_ids"][0]
# print("Type: ", type(example))
# print("Length: ", len(example))
# print(example)
# tokenizer.decode(example)

In [13]:
# whole word masking: want to mask all the tokens that correspond to a single word. 

import collections
import numpy as np

from transformers import default_data_collator

def whole_word_masking_data_collator(features, wwm_prob=0.15):
    
    for feature in features:
        word_ids = feature.pop("word_ids") #to fit into default_data_collator
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word: # removing repeat word_ids 
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)
            #created a list where each index is a whole word, as a list of the indices of it's tokens
            
        mask = np.random.binomial(1, wwm_prob, (len(mapping),))#each whole word rather than each token has equal chance of selection for masking

        input_ids = feature['input_ids']
        #print(f'Length of input_ids is  {len(input_ids)}')
        labels = feature['labels']
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id    
                
    return default_data_collator(features)

## Masking Padded Example:

In [14]:
example_samples = [padded_dataset["train"][i] for i in range(1)]
for sample in example_samples:
    # THIS IS A FIX - THE PADDED DATA IS MISSING LABELS
    sample["labels"] = sample["input_ids"].copy()
example_batch = whole_word_masking_data_collator(example_samples)

for chunk in example_batch["input_ids"]:
    a = tokenizer.convert_ids_to_tokens(chunk)
    print(f"\n'>>> {tokenizer.convert_ids_to_tokens(chunk)}'")


'>>> ['[CLS]', 'final', '[MASK]', '[MASK]', 'chest', 'p', '##a', 'and', 'la', '##t', '[MASK]', 'f', '[MASK]', 'new', 'onset', 'as', '##cite', '##s', 'e', '##val', 'for', 'infection', 'technique', '[MASK]', 'p', '##a', 'and', 'lateral', 'comparison', '[MASK]', '[MASK]', 'there', 'is', 'no', 'focal', 'consolidation', 'p', '##le', '##ural', 'e', '##ff', '##usion', 'or', 'p', '##ne', '##um', '##oth', '##orax', '[MASK]', '[MASK]', '[MASK]', 'op', '##ac', '##ities', 'that', '[MASK]', '[MASK]', 'represent', 'nipple', '[MASK]', 'the', 'card', '##io', '##media', '##st', '##inal', 'silhouette', 'is', 'normal', '[MASK]', 'project', 'over', 'the', 'left', 'lung', 'potentially', 'within', '[MASK]', 'breast', '[MASK]', '[MASK]', '[MASK]', 'upper', 'abdomen', 'is', 'un', '##rem', '##ark', '##able', 'chronic', '[MASK]', '[MASK]', '[MASK]', 'of', 'the', 'posterior', 'left', 'sixth', 'and', 'seventh', 'ribs', 'are', 'noted', 'impression', 'no', '[MASK]', 'card', '##io', '##pu', '##lm', '##ona', '##ry',

# Model Training (Padding)
### 1. Prepare the datasets batches (with whole-word-masking) 

In [15]:
# Eliminate this source of randomness is to apply the masking once on the whole test set,
# and then use the default data collator in 🤗 Transformers to collect the batches during evaluation


# replace data_collator here with the whole-word-masking ones
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = whole_word_masking_data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [16]:
paddedDataset = padded_dataset.remove_columns(['word_ids'])
paddedDataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 221006
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
})

In [17]:
# Apply this function to our test set and drop the unmasked columns so we can replace them with the masked ones
eval_paddedDataset = padded_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=padded_dataset["test"].column_names,
)
eval_paddedDataset = eval_paddedDataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_token_type_ids": "token_type_ids", 
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)
tokenizer.decode(eval_paddedDataset["input_ids"][0])

Loading cached processed dataset at /media/SharedUsers/dlc19/home/.cache/huggingface/datasets/csv/default-cd976364fa85dbba/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-567e2d642cf3dd59.arrow


'[CLS] final report ap chest [MASK] history intubated yearold woman check tube placement [MASK] ap chest compared to [MASK] [MASK] the endotracheal [MASK] at the upper [MASK] of the [MASK] [MASK] [MASK] [MASK] [MASK] less than mm [MASK] [MASK] carina care should be taken that [MASK] tube does [MASK] withdraw [MASK] further lungs [MASK] [MASK] cardiomediastinal and [MASK] [MASK] silhouettes and pleural surfaces are normal [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [18]:
torch.cuda.empty_cache()

### 2. Set up dataloaders

In [19]:
batch_size = 64

train_dataloader = DataLoader(
    padded_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=whole_word_masking_data_collator, # replace self-defined whole-word-masking-data-collator
)

# Use the default_data_collator from Transformers for the evaluation set
eval_dataloader = DataLoader(
    eval_paddedDataset, 
    batch_size=batch_size, 
    collate_fn=default_data_collator
)

In [20]:
# for batch_idx, data in enumerate(train_dataloader):
#     print(batch_idx)
#     print(data["input_ids"])

### 3. Steps for training with Accelerate

In [21]:
# Is it correct lolllll?
model = AutoModelForMaskedLM.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# https://huggingface.co/docs/transformers/model_doc/auto

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
# Adam optimizer 
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [23]:
from accelerate import Accelerator

accelerator = Accelerator(cpu=True)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, 
    optimizer, 
    train_dataloader, 
    eval_dataloader
)

In [24]:
# Learning rate scheduler:
from transformers import get_scheduler

num_train_epochs = 10 # change this later

num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [25]:
######### Saving onto Huggingface hub ###########
from huggingface_hub import get_full_repo_name

model_name = "CXR_BioClinicalBERT_SW"
repo_name = get_full_repo_name(model_name)
repo_name

'ICLbioengNLP/CXR_BioClinicalBERT_SW'

In [26]:
from huggingface_hub import Repository

output_dir = model_name
repo = Repository(output_dir, clone_from=repo_name)

/media/SharedUsers/dlc19/home/codes/nlp-fineTuningBERT/CXR_BioClinicalBERT_SW is already a clone of https://huggingface.co/ICLbioengNLP/CXR_BioClinicalBERT_SW. Make sure you pull the latest changes with `repo.git_pull()`.


### 5. Full Training Loop

In [ ]:
from tqdm.auto import tqdm
import math

progress_bar = tqdm(range(num_training_steps))
perplexities = []

for epoch in range(num_train_epochs): 
    # Training
    model.train()
    
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    print("Finishing epoch ", epoch)
    
    # Evaluation
    model.eval()
    losses = []
    
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_chunkedDataset)]
    
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")
    perplexities.append(perplexity)

    
    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/34540 [00:00<?, ?it/s]

############# Epoch 0: Training Start   #############


In [90]:
print(perplexities)
print(losses)

[1.1576959602969485, 1.1498089768602706, 1.1442581128031095, 1.1400363561367657, 1.1363503719807606, 1.13433012291388, 1.132400135905862, 1.1313966997247378, 1.1313809281236062, 1.1313809281236062]
tensor([0.1343, 0.1343, 0.1343,  ..., 0.1029, 0.1029, 0.1029], device='cuda:0')


# Comparing pre-trained and fine-tuned models
#### Can also do BLEU score evaluation?

In [3]:
from transformers import pipeline

mask_filler_ft = pipeline(
    "fill-mask", model="ICLbioengNLP/CXR_BioClinicalBERT_SW")

mask_filler_original = pipeline(
    "fill-mask", model="emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
text1 = "There are no signs of [MASK]."

preds1_ft = mask_filler_ft(text1)
preds1_org = mask_filler_original(text1)

print("Predictions for text1:")
print("1. Pre-trained Bio_ClinicalBERT")
for pred in preds1_org:
    print(f">>> {pred['sequence']}")
print()
print("2. Fine-tuned CXR_BioClinicalBERT_SW")
for pred in preds1_ft:
    print(f">>> {pred['sequence']}")

Predictions for text1:
1. Pre-trained Bio_ClinicalBERT
>>> there are no signs of bleeding.
>>> there are no signs of infection.
>>> there are no signs of withdrawal.
>>> there are no signs of change.
>>> there are no signs of distress.

2. Fine-tuned CXR_BioClinicalBERT_chunkedv1
>>> there are no signs of pneumonia.
>>> there are no signs of failure.
>>> there are no signs of consolidation.
>>> there are no signs of congestion.
>>> there are no signs of infection.


In [5]:
text2 = "The patient suffered from [MASK]. "
preds2_ft = mask_filler_ft(text2)
preds2_org = mask_filler_original(text2)

print("Predictions for text2:")
print("1. Pre-trained Bio_ClinicalBERT")
for pred in preds2_org:
    print(f">>> {pred['sequence']}")
print()
print("2. Fine-tuned CXR_BioClinicalBERT_SW")
for pred in preds2_ft:
    print(f">>> {pred['sequence']}")

Predictions for text2:
1. Pre-trained Bio_ClinicalBERT
>>> the patient suffered from pneumonia.
>>> the patient suffered from anxiety.
>>> the patient suffered from fatigue.
>>> the patient suffered from fall.
>>> the patient suffered from this.

2. Fine-tuned CXR_BioClinicalBERT_chunkedv1
>>> the patient suffered from pneumonia.
>>> the patient suffered from trauma.
>>> the patient suffered from bleeding.
>>> the patient suffered from seizure.
>>> the patient suffered from surgery.
